In [2]:
import pandas as pd
import requests
import pprint
import json
import time
from pandas import json_normalize
import ast

pd.set_option('display.max_colwidth', None) ## 셀 최대 너비 제한 해제

In [3]:
# api_key 저장
api_key = 'RGAPI-ca44b272-60ec-4589-8980-8f28e65ca212' ## 만료되면 갱신해줘야함 거의 하루마다

# request header 저장
request_header = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36",
    "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
    "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
    "Origin": "https://developer.riotgames.com",
    "X-Riot-Token": api_key 
    }

# base_url 저장
base_url = "https://kr.api.riotgames.com/tft/" ## 이 뒤에다가 + 로 그때그떄 필요한 URL 을 더해줄 것

##### 앞에서 만든 게임 아이디 테이블 불러오기

In [4]:
gameId_df = pd.read_csv('C:/Users/user/Documents/GitHub/Final-Project-A09-/tier_and_gameId.csv')

gameId_df.head(3)   

,tier,gameId
0,CHALLENGER,KR_7005435650
1,CHALLENGER,KR_7005408175
2,CHALLENGER,KR_7005214080


#### 게임 아이디 테이블의 모든 게임 결과 데이터 불러오기

##### 하나의 게임 아이디로부터 게임 결과 데이터를 불러오는 쿼리 작성 테스트

In [5]:
gameId = 'KR_7003200600'

# gameId 를 통해 game result data 를 불러오는 API URL 설정
game_result_url = "https://Asia.api.riotgames.com/tft/" + f"match/v1/matches/{gameId}"

# 조회한 결과를 game_result 에 저장
game_result = requests.get(game_result_url, headers=request_header).json()

# 게임 데이터 중 실질적으로 중요한 정보를 담고있는 info 컬럼을 데이터 프레임으로 저장
game_result_df = pd.DataFrame(game_result['info'])

# 반환된 게임 결과 데이터 프레임의 gameId 컬럼에 KR_ 를 붙여주는 것으로 tier_gameId 테이블과 컬럼값을 동일하게
game_result_df['gameId'] = 'KR_' + game_result_df['gameId'].astype(str)

# 필요없는 컬럼 drop
game_result_df = game_result_df[['gameId', 'game_datetime', 'game_length', 'game_version', 'participants']]

In [6]:
game_result_df.head(1)

gameId  game_datetime  game_length  \
0  KR_7003200600  1711447615823   2141.46582   

                                                            game_version  \
0  Version 14.6.570.6276 (Mar 25 2024/14:00:15) [PUBLIC] <Releases/14.6>   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

##### 모든 게임 아이디에 대한 게임 결과 데이터를 조회하여 하나의 테이블로 만들기

In [8]:
import time

# gameId 목록 추출
game_ids = list(gameId_df['gameId'])

# 빈 데이터 프레임 생성 (초기에는 데이터 없음)
game_results_df = pd.DataFrame()

# 요청 제한을 고려하여 요청 속도 조절
requests_per_minute = 70  # 2분에 허용된 요청 수
requests_per_second = requests_per_minute / 70  # 1초에 허용된 요청 수
interval = 1 / requests_per_second  # 각 요청 사이의 시간 간격

# 각 게임 아이디에 대한 게임 결과 정보 조회 및 저장
for game_id in game_ids:
    game_result_url = "https://Asia.api.riotgames.com/tft/" + f"match/v1/matches/{game_id}"
    game_result = requests.get(game_result_url, headers=request_header).json()

    # Check if 'info' key exists
    if 'info' in game_result:
        # Create a temporary DataFrame from game_result['info']
        temp_df = pd.DataFrame(game_result['info'])
        # Concatenate the temporary DataFrame with game_results_df
        game_results_df = pd.concat([game_results_df, temp_df], ignore_index=True)
    else:
        # Handle missing information (log a warning or assign a default value)
        print(f"WARNING: 'info' key not found for game ID: {game_id}")

    # 요청 사이에 일정한 시간 간격을 두기 위해 대기
    time.sleep(interval)

# 필요없는 컬럼 drop
game_results_df = game_results_df[['gameId', 'game_datetime', 'game_length', 'game_version', 'participants']]

# gameId 컬럼에 'KR_' 추가
game_results_df['gameId'] = 'KR_' + game_results_df['gameId'].astype(str)

In [9]:
# 결과 확인
game_results_df

gameId  game_datetime  game_length  \
0     KR_7005435650  1711611092537  2339.676514   
1     KR_7005435650  1711611092537  2339.676514   
2     KR_7005435650  1711611092537  2339.676514   
3     KR_7005435650  1711611092537  2339.676514   
4     KR_7005435650  1711611092537  2339.676514   
...             ...            ...          ...   
9283  KR_7005139736  1711571877752  2173.421143   
9284  KR_7005139736  1711571877752  2173.421143   
9285  KR_7005139736  1711571877752  2173.421143   
9286  KR_7005139736  1711571877752  2173.421143   
9287  KR_7005139736  1711571877752  2173.421143   

                                                               game_version  \
0     Version 14.6.570.6276 (Mar 25 2024/14:00:15) [PUBLIC] <Releases/14.6>   
1     Version 14.6.570.6276 (Mar 25 2024/14:00:15) [PUBLIC] <Releases/14.6>   
2     Version 14.6.570.6276 (Mar 25 2024/14:00:15) [PUBLIC] <Releases/14.6>   
3     Version 14.6.570.6276 (Mar 25 2024/14:00:15) [PUBLIC] <Releases/14.6>   
4     Version 14.6.570.6276 (Mar 25 2024/14:00:15) [PUBLIC] <Releases/14.6>   
...                                                                     ...   
9283  Version 14.6.570.6276 (Mar 25 2024/14:00:15) [PUBLIC] <Releases/14.6>   
9284  Version 14.6.570.6276 (Mar 25 2024/14:00:15) [PUBLIC] <Releases/14.6>   
9285  Version 14.6.570.6276 (Mar 25 2024/14:00:15) [PUBLIC] <Releases/14.6>   
9286  Version 14.6.570.6276 (Mar 25 2024/14:00:15) [PUBLIC] <Releases/14.6>   
9287  Version 14.6.570.6276 (Mar 25 2024/14:00:15) [PUBLIC] <Releases/14.6>   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   

##### game_result_df 의 participants 컬럼을 풀어서 컬럼화 시키기전 csv 파일로 한번 저장

In [10]:
game_results_df.to_csv('game_results_raw_data.csv', index=False)

##### game_result_df 로부터 필요한 컬럼의 json 형식을 normalize 하여 테이블화

In [11]:
# 저장한 csv 파일 가져오기
game_results_df = pd.read_csv('C:/Users/user/Documents/GitHub/Final-Project-A09-/game_results_raw_data.csv')

In [12]:
# 'participants' 컬럼의 데이터 타입 확인
print(game_results_df['participants'].dtype)

object


##### 데이터 타입이 object 이므로 json(dictionary) 형식으로 변환

In [13]:
# 'participants' 열의 데이터를 JSON으로 변환하여 사전 형태로 저장하는 함수
def parse_participants(participants_str):
    try:
        participants_dict = json.loads(participants_str.replace("'", "\"")) # JSON 형식으로 변환하고 따옴표를 쌍 따옴표로 변경
        return participants_dict
    except json.JSONDecodeError:
        return None

##### 변환해준 테이블을 normalize 해준 새로운 테이블 생성

In [14]:
# 새로운 데이터프레임 생성
participants_df = game_results_df[['gameId', 'game_datetime', 'game_length', 'game_version']].copy()


# 'participants' 열의 데이터를 JSON으로 변환하여 사전 형태로 저장
participants_df['participants_dict'] = game_results_df['participants'].apply(parse_participants)


## 'participants_dict' 열에서 필요한 정보 추출하여 새로운 열 생성
# 'participants' 컬럼의 데이터에서 'puuid' 값을 추출하여 새로운 열 생성  # 유저 식별자

participants_df['puuid'] = participants_df['participants_dict'].apply(lambda x: x['puuid'] if isinstance(x, dict) and 'puuid' in x else None)

# 'placement'  # 최종 등수

participants_df['placement'] = participants_df['participants_dict'].apply(lambda x: x['placement'] if isinstance(x, dict) and 'placement' in x else None)

# 'last_round'  # 버틴 라운드 수

participants_df['last_round'] = participants_df['participants_dict'].apply(lambda x: x['last_round'] if isinstance(x, dict) and 'last_round' in x else None)

# 'level'  # 경기 종료 시 플레이어 레벨

participants_df['level'] = participants_df['participants_dict'].apply(lambda x: x['level'] if isinstance(x, dict) and 'level' in x else None)

# 'Augments'  # 경기 종료 시 보유한 증강체

participants_df['augments'] = participants_df['participants_dict'].apply(lambda x: x['augments'] if isinstance(x, dict) and 'augments' in x else None)

# 'traits'  # 경기 종료 시 활성화된 시너지

participants_df['traits'] = participants_df['participants_dict'].apply(lambda x: x['traits'] if isinstance(x, dict) and 'traits' in x else None)

# 'units'  # 경기 종료시 가지고 있는 유닛, 장착된 아이템

participants_df['units'] = participants_df['participants_dict'].apply(lambda x: x['units'] if isinstance(x, dict) and 'units' in x else None)

# 'participants_dict' 열은 더 이상 필요 없으므로 제거

participants_df.drop(columns=['participants_dict'], inplace=True)

# 결과 확인
participants_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9288 entries, 0 to 9287
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   gameId         9288 non-null   object 
 1   game_datetime  9288 non-null   int64  
 2   game_length    9288 non-null   float64
 3   game_version   9288 non-null   object 
 4   puuid          9288 non-null   object 
 5   placement      9288 non-null   int64  
 6   last_round     9288 non-null   int64  
 7   level          9288 non-null   int64  
 8   augments       9288 non-null   object 
 9   traits         9288 non-null   object 
 10  units          9288 non-null   object 
dtypes: float64(1), int64(4), object(6)
memory usage: 798.3+ KB


##### 최종 결과 확인 및  csv 파일로 저장

In [15]:
# 새로운 데이터프레임 생성
participants_df = game_results_df[['gameId', 'game_datetime', 'game_length', 'game_version']].copy()


# 'participants' 열의 데이터를 JSON으로 변환하여 사전 형태로 저장
participants_df['participants_dict'] = game_results_df['participants'].apply(parse_participants)


## 'participants_dict' 열에서 필요한 정보 추출하여 새로운 열 생성
# 'participants' 컬럼의 데이터에서 'puuid' 값을 추출하여 새로운 열 생성  # 유저 식별자

participants_df['puuid'] = participants_df['participants_dict'].apply(lambda x: x['puuid'] if isinstance(x, dict) and 'puuid' in x else None)

# 'placement'  # 최종 등수

participants_df['placement'] = participants_df['participants_dict'].apply(lambda x: x['placement'] if isinstance(x, dict) and 'placement' in x else None)

# 'last_round'  # 버틴 라운드 수

participants_df['last_round'] = participants_df['participants_dict'].apply(lambda x: x['last_round'] if isinstance(x, dict) and 'last_round' in x else None)

# 'level'  # 경기 종료 시 플레이어 레벨

participants_df['level'] = participants_df['participants_dict'].apply(lambda x: x['level'] if isinstance(x, dict) and 'level' in x else None)

# 'Augments'  # 경기 종료 시 보유한 증강체

participants_df['augments'] = participants_df['participants_dict'].apply(lambda x: x['augments'] if isinstance(x, dict) and 'augments' in x else None)

# 'traits'  # 경기 종료 시 활성화된 시너지

participants_df['traits'] = participants_df['participants_dict'].apply(lambda x: x['traits'] if isinstance(x, dict) and 'traits' in x else None)

# 'units'  # 경기 종료시 가지고 있는 유닛, 장착된 아이템

participants_df['units'] = participants_df['participants_dict'].apply(lambda x: x['units'] if isinstance(x, dict) and 'units' in x else None)

# 'participants_dict' 열은 더 이상 필요 없으므로 제거

participants_df.drop(columns=['participants_dict'], inplace=True)

# 결과 확인
participants_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9288 entries, 0 to 9287
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   gameId         9288 non-null   object 
 1   game_datetime  9288 non-null   int64  
 2   game_length    9288 non-null   float64
 3   game_version   9288 non-null   object 
 4   puuid          9288 non-null   object 
 5   placement      9288 non-null   int64  
 6   last_round     9288 non-null   int64  
 7   level          9288 non-null   int64  
 8   augments       9288 non-null   object 
 9   traits         9288 non-null   object 
 10  units          9288 non-null   object 
dtypes: float64(1), int64(4), object(6)
memory usage: 798.3+ KB


In [16]:
# csv 파일로 저장
participants_df.to_csv('game_results_of_each_participants.csv', index=False)

In [17]:
participants_df

,gameId,game_datetime,game_length,game_version,puuid,placement,last_round,level,augments,traits,units
0,KR_7005435650,1711611092537,2339.676514,Version 14.6.570.6276 (Mar 25 2024/14:00:15) [PUBLIC] <Releases/14.6>,BWLn3nfAuki-3ju22n-ni5exBESPlXaPsyF9bZ3uFeuvr1ks-ZL6B7x3SA_KVt_kNrHRfsg6VVk9YQ,7,28,8,"[TFT9_Augment_IronAssets, TFT9_Augment_YouHaveMyBow, TFT6_Augment_PortableForgePlusPlus]","[{'name': 'TFT11_Altruist', 'num_units': 1, 'style': 0, 'tier_current': 0, 'tier_total': 3}, {'name': 'TFT11_Behemoth', 'num_units': 2, 'style': 1, 'tier_current': 1, 'tier_total': 3}, {'name': 'TFT11_Bruiser', 'num_units': 4, 'style': 2, 'tier_current': 2, 'tier_total': 4}, {'name': 'TFT11_Exalted', 'num_units': 3, 'style': 2, 'tier_current': 1, 'tier_total': 2}, {'name': 'TFT11_Ghostly', 'num_units': 1, 'style': 0, 'tier_current': 0, 'tier_total': 4}, {'name': 'TFT11_InkShadow', 'num_units': 2, 'style': 0, 'tier_current': 0, 'tier_total': 3}, {'name': 'TFT11_Mythic', 'num_units': 1, 'style': 0, 'tier_current': 0, 'tier_total': 4}, {'name': 'TFT11_SpiritWalker', 'num_units': 1, 'style': 5, 'tier_current': 1, 'tier_total': 1}, {'name': 'TFT11_Storyweaver', 'num_units': 3, 'style': 1, 'tier_current': 1, 'tier_total': 4}, {'name': 'TFT11_Trickshot', 'num_units': 2, 'style': 1, 'tier_current': 1, 'tier_total': 2}, {'name': 'TFT11_Umbral', 'num_units': 1, 'style': 0, 'tier_current': 0, 'tier_total': 4}]","[{'character_id': 'TFT11_Sivir', 'itemNames': [], 'name': '', 'rarity': 0, 'tier': 2}, {'character_id': 'TFT11_Riven', 'itemNames': [], 'name': '', 'rarity': 1, 'tier': 2}, {'character_id': 'TFT11_Shen', 'itemNames': [], 'name': '', 'rarity': 1, 'tier': 1}, {'character_id': 'TFT11_TahmKench', 'itemNames': ['TFT4_Item_OrnnRanduinsSanctum'], 'name': '', 'rarity': 2, 'tier': 2}, {'character_id': 'TFT11_Galio', 'itemNames': ['TFT_Item_Crownguard', 'TFT_Item_WarmogsArmor', 'TFT_Item_AdaptiveHelm'], 'name': '', 'rarity': 4, 'tier': 1}, {'character_id': 'TFT11_Kaisa', 'itemNames': ['TFT_Item_GuinsoosRageblade', 'TFT_Item_RapidFireCannon', 'TFT9_Item_OrnnHorizonFocus'], 'name': '', 'rarity': 4, 'tier': 2}, {'character_id': 'TFT11_Sylas', 'itemNames': ['TFT_Item_Bloodthirster', 'TFT_Item_UnstableConcoction', 'TFT_Item_LocketOfTheIronSolari'], 'name': '', 'rarity': 4, 'tier': 2}, {'character_id': 'TFT11_Udyr', 'itemNames': [], 'name': '', 'rarity': 6, 'tier': 1}, {'character_id': 'TFT11_Kayle', 'itemNames': ['TFT11_Storyweaver3_Support'], 'name': '', 'rarity': 9, 'tier': 1}]"
1,KR_7005435650,1711611092537,2339.676514,Version 14.6.570.6276 (Mar 25 2024/14:00:15) [PUBLIC] <Releases/14.6>,mptEuTzeZ2MWsLxe4KfwLTr2iJp2uUphE7maw8CwGfeeFKXOVZ0_E8GccCzgNUfy0wiRJ1BGd1LOGQ,1,40,9,"[TFT9_Augment_CustomerIsAlwaysRight, TFT9_Augment_Martyr, TFT9_Augment_Commander_Ascension]","[{'name': 'TFT11_Behemoth', 'num_units': 1, 'style': 0, 'tier_current': 0, 'tier_total': 3}, {'name': 'TFT11_Bruiser', 'num_units': 2, 'style': 1, 'tier_current': 1, 'tier_total': 4}, {'name': 'TFT11_Dryad', 'num_units': 6, 'style': 3, 'tier_current': 3, 'tier_total': 3}, {'name': 'TFT11_Fated', 'num_units': 1, 'style': 0, 'tier_current': 0, 'tier_total': 4}, {'name': 'TFT11_Ghostly', 'num_units': 2, 'style': 1, 'tier_current': 1, 'tier_total': 4}, {'name': 'TFT11_InkShadow', 'num_units': 3, 'style': 1, 'tier_current': 1, 'tier_total': 3}, {'name': 'TFT11_Invoker', 'num_units': 1, 'style': 0, 'tier_current': 0, 'tier_total': 3}, {'name': 'TFT11_Reaper', 'num_units': 1, 'style': 0, 'tier_current': 0, 'tier_total': 2}, {'name': 'TFT11_Sniper', 'num_units': 2, 'style': 1, 'tier_current': 1, 'tier_total': 3}, {'name': 'TFT11_Warden', 'num_units': 2, 'style': 1, 'tier_current': 1, 'tier_total': 3}]","[{'character_id': 'TFT11_Caitlyn', 'itemNames': [], 'name': '', 'rarity': 0, 'tier': 3}, {'character_id': 'TFT11_RekSai', 'itemNames': ['TFT_Item_FrozenHeart', 'TFT_Item_NightHarvester'], 'name': '', 'rarity': 0, 'tier': 2}, {'character_id': 'TFT11_Jax', 'itemNames': ['TFT11_Item_DryadEmblemItem', 'TFT_It